# Setting up a mock DB to store Songs, users, and playlists
![](https://github.com/spotify-song/DS/blob/master/api/data/DB_tables_illu.png?raw=true)
### Tables:
    - User:
        - id
        - display_name
    - Tokens:
        - id
        - access_token
        - token_type
        - expries_in
        - refresh_token
        - scope
        - expires_at
        - user
    - Tracks
        - id
        - danceability
        - energy
        - loudness
        - mode
        - speechiness
        - accousticness
        - instrumentalness
        - valence
        - liveness
        - tempo
        - duration_ms
        - time_signature
    - UserPlaylist
        - id
        - u_id
        - uri
        - tracks_id

## Importing db classes for tables

In [1]:
import sys
sys.path.insert(0, '../')
import models.my_db
from models.my_db import *
from spotify_users import UserData

## DB import flow needs to be:
    - User > tokens
    - Tracks > top_tracks
    - users/tracks > user_playlists

### Get the pertinent data using the UserData() class

In [2]:
user = UserData()
user_dict = user.user_top_50(user_id='Agustinvargas')

In [3]:
spot_cc = user_dict['spotify connect']
current_user_info = user_dict['Current User Info']
token_info = user_dict['Tokens Info']
top_50_trx_ids = user_dict['Top Track IDs']
top_50_aud_feat = user_dict['Track Audio Features']

In [4]:
session, user1 = user.check_for_user(current_user_info=current_user_info, spot_cc=spot_cc)

User Agustinvargas already exists in DataBase;                    checking for token
Agustinvargas has a token, updating it




In [5]:
session2 = user.add_tracks(top_50_trx_ids=top_50_trx_ids, top_50_aud_feat=top_50_aud_feat, session=session)

In [4]:
# Token details
# user_dict['Tokens Info']['access_token']
# user_dict['Tokens Info']['token_type']
# user_dict['Tokens Info']['expires_in']
# user_dict['Tokens Info']['scope']
# user_dict['Tokens Info']['expires_at']
# user_dict['Tokens Info']['refresh_token']

# Data for a given user: Token, ID, Display Name, Top 50 tracks, Track Audio Features
# print(user_dict['User ID'],
#       user_dict['Display Name'],
#       user_dict['Top Track IDs'],
#       user_dict['Track Audio Features'])


# Track audio features (the dict contains audio feats for all 50 tracks)
print(
    user_dict['Track Audio Features'][1]['danceability'],
    user_dict['Track Audio Features'][1]['energy'],
    user_dict['Track Audio Features'][1]['key'],
    user_dict['Track Audio Features'][1]['loudness'],
    user_dict['Track Audio Features'][1]['mode'],
    user_dict['Track Audio Features'][1]['speechiness'],
    user_dict['Track Audio Features'][1]['acousticness'],
    user_dict['Track Audio Features'][1]['instrumentalness'],
    user_dict['Track Audio Features'][1]['valence'],
    user_dict['Track Audio Features'][1]['liveness'],
    user_dict['Track Audio Features'][1]['tempo'],
    user_dict['Track Audio Features'][1]['duration_ms'],
    user_dict['Track Audio Features'][1]['time_signature']
    )

0.726 0.125 5 -9.194 0 0.0803 0.835 0 0.277 0.131 92.23 183711 4


In [5]:
# Create session
session = user.Session()

### First we create/find a user

### Instantiate token add the user1 data

In [6]:
# instantiating User() object corresponding to user table in DB

user1 = User(id=user_dict['User ID'], display_name=user_dict['Display Name'])

### Instantiate a track

In [7]:
print('importing a track into the database for a given user and play list')
# trk2 = Tracks(
#             id=user_dict['Top Track IDs'][1],
#             danceability=user_dict['Track Audio Features'][1]['danceability'],
#             energy=user_dict['Track Audio Features'][1]['energy'],
#             key=user_dict['Track Audio Features'][1]['key'],
#             loudness=user_dict['Track Audio Features'][1]['loudness'],
#             mode=user_dict['Track Audio Features'][1]['mode'],
#             speechiness=user_dict['Track Audio Features'][1]['speechiness'],
#             acousticness=user_dict['Track Audio Features'][1]['acousticness'],
#             instrumentalness=user_dict['Track Audio Features'][1]['instrumentalness'],
#             valence=user_dict['Track Audio Features'][1]['valence'],
#             liveness=user_dict['Track Audio Features'][1]['liveness'],
#             tempo=user_dict['Track Audio Features'][1]['tempo'],
#             duration_ms=user_dict['Track Audio Features'][1]['duration_ms'],
#             time_signature=user_dict['Track Audio Features'][1]['time_signature']
#             )

importing a track into the database for a given user and play list


### Checking to see if song exists in DB if not then add it to db

In [27]:
trk_sesh = session.query(Tracks).filter(Tracks.id == trk2.id).first()
if trk_sesh is None:
    print(f"track with id {trk2.id} does not exists, so we'll up date it")
    session.add(trk2)
    session.commit()
else:
    print(f"The song with id {trk2.id} is already in our DB")

track with id 02gaYAEdeR6poHcBH1KUQF does not exists, so we'll up date it


### Creating `Tracks` objects, and adding to DB using `session.add_all()`

# Completed Functions for the `UserData` class
    Functions completed:
        - UserData().check_for_user(): checks if user exists in DB if
        - UserData().add_tracks(): checks to see if tracks exist in DB if they don't it updates them

In [8]:
def check_for_user(self, current_user_info, spot_cc):
    """
    Checks for user in DB.

    Input:
        - current_user_info: This object contains ID and displa_name for
          the user
        - spot_cc: Object for the spotify credentials connection

    Output:
        - : User() object from the my_db module
             contains q.id (user id) and q.display_name (user display name)
    """
    session = self.Session()
    engine = self.engine
    Base = self.base
    current_user_info = current_user_info
    spot_cc = spot_cc

    user1 = User(
                id=current_user_info['id'],
                display_name=current_user_info['display_name']
                )

    # update user and commit, then update token, and commit
    display_name_q = session.query(
                                    User
                                    ).filter(
                                            User.display_name ==
                                            user1.display_name
                                            ).first()

    # checking if user exists, if not then add user and token
    # if exists, check if token exists, if not add
    # if token exists, update using refresh_key
    if display_name_q is None:
        session.add(user1)
        session.commit()
        token = Tokens(
                    # Instead of of "user_dict['Tokens Info']"" it should
                    # be "token_info"
                    access_token=token_info['access_token'],
                    token_type=token_info['token_type'],
                    expires_in=token_info['expires_in'],
                    refresh_token=token_info['refresh_token'],
                    scope=token_info['scope'],
                    expires_at=token_info['expires_at'],
                    # Still user "user1" because the id is not changing,
                    # simply adding mans to the db
                    user_token=user1
                    )
        session.add(token)
        session.commit()
    # Else, check if user has corresponding token that is not expired
    else:
        print(f"User {user1.display_name} already exists in DataBase;\
                checking for token")
        token_q = session.query(
                                Tokens
                                ).filter(
                                        Tokens.user == user1.id
                                        ).first()
        if token_q is None:
            print(f"We don't have a token for {user1.display_name},\
                    adding new token")
            token = Tokens(
                        # "token_info" to add tokens to db
                        access_token=token_info['access_token'],
                        token_type=token_info['token_type'],
                        expires_in=token_info['expires_in'],
                        refresh_token=token_info['refresh_token'],
                        scope=token_info['scope'],
                        expires_at=token_info['expires_at'],
                        # Still user "user1" because the id is not changing
                        # simply adding mans to the db
                        user_token=user1
                        )
            session.add(token)
            session.commit()
        # if the token exists refresh it, and update the value in table
        else:
            print(f"{user1.display_name} has a token, updating it")
            accs_token_refresh = spot_cc.refresh_access_token(
                                                    token_q.refresh_token
                                                    )
            token_info = spot_cc.get_access_token(as_dict=True)
            update_token = update(Tokens).where(
                                            Tokens.user == user1.id
                                                ).values(
                                                access_token=token_info[
                                                            'access_token'
                                                                ],
                                                expires_in=token_info[
                                                            'expires_in'
                                                                ],
                                                refresh_token=token_info[
                                                            'refresh_token'
                                                                ],
                                                expires_at=token_info[
                                                            'expires_at'
                                                                ]
                                                        )
            session.execute(update_token)
            session.commit()

    return session, user1

In [6]:
def add_tracks(self, top_50_trx_ids, top_50_aud_feat, session):
    """
    This function is used when the user does not have their top 50 trx in
    our db, and we need to add them after adding the user.

    Input:
        - top_50_trx_ids: List of 50 alpha-neumeric track IDs
        - top_50_aud_feat: List of dicts containing feat data

    Output:
        - Updates DB with the track id and track feature data
        - DB session: object
    """
    top_50_aud_feat = top_50_aud_feat
    top_50_trx_ids = top_50_trx_ids
    session = session

    trx = []
    for k, v in enumerate(top_50_aud_feat):
        trk_sesh = session.query(
                                Tracks
                                ).filter(
                                        Tracks.id == top_50_trx_ids[k]
                                        ).first()
        if trk_sesh is not None:
            continue
        globals()['trk_' + str(k)] = Tracks(
                                    id=top_50_trx_ids[k],
                                    danceability=v['danceability'],
                                    energy=v['energy'],
                                    key=v['key'],
                                    loudness=v['loudness'],
                                    mode=v['mode'],
                                    speechiness=v['speechiness'],
                                    acousticness=v['acousticness'],
                                    instrumentalness=v['instrumentalness'],
                                    valence=v['valence'],
                                    liveness=v['liveness'],
                                    tempo=v['tempo'],
                                    duration_ms=v['duration_ms'],
                                    time_signature=v['time_signature']
                                    )
        trx.append(globals()['trk_' + str(k)])

    session.add_all(trx)
    session.commit()

    return session